In [1]:
import os
import re
import json
import requests 
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup

In [2]:
if not os.path.isdir('data/'):
    os.mkdir('data/')

# Understat.com

Followed this Kaggle [tutorial](https://www.kaggle.com/slehkyi/web-scraping-football-statistics-2014-now#Scraping-data-for-all-teams-of-all-leagues-of-all-seasons).

In [3]:
data_path = 'data/understat/'

if not os.path.isdir(data_path):
    os.mkdir(data_path)

In [4]:
# create urls for all seasons of all leagues
base_url = 'https://understat.com/league'
leagues = ['La_liga', 'EPL', 'Bundesliga', 'Serie_A', 'Ligue_1']
seasons = ['2014', '2015', '2016', '2017', '2018', '2019', '2020']

In [5]:
full_data = dict()
for league in leagues:
  
    season_data = dict()
    for season in seasons:    
        url = base_url+'/'+league+'/'+season
        res = requests.get(url)
        soup = BeautifulSoup(res.content, "html.parser")

        # Based on the structure of the webpage, I found that data is in the JSON variable, under <script> tags
        scripts = soup.find_all('script')

        string_with_json_obj = ''

        # Find data for teams
        for el in scripts:
            if 'teamsData' in str(el):
                string_with_json_obj = str(el).strip()

        # print(string_with_json_obj)

        # strip unnecessary symbols and get only JSON data
        ind_start = string_with_json_obj.index("('")+2
        ind_end = string_with_json_obj.index("')")
        json_data = string_with_json_obj[ind_start:ind_end]
        json_data = json_data.encode('utf8').decode('unicode_escape')


        # convert JSON data into Python dictionary
        data = json.loads(json_data)

        # Get teams and their relevant ids and put them into separate dictionary
        teams = {}
        for id in data.keys():
            teams[id] = data[id]['title']

        # EDA to get a feeling of how the JSON is structured
        # Column names are all the same, so we just use first element
        columns = []
        # Check the sample of values per each column
        values = []
        for id in data.keys():
            columns = list(data[id]['history'][0].keys())
            values = list(data[id]['history'][0].values())
            break

        # Getting data for all teams
        dataframes = {}
        for id, team in teams.items():
            teams_data = []
            for row in data[id]['history']:
                teams_data.append(list(row.values()))

            df = pd.DataFrame(teams_data, columns=columns)
            dataframes[team] = df

        for team, df in dataframes.items():
            dataframes[team]['ppda_coef'] = \
                dataframes[team]['ppda'].apply(lambda x: x['att']/x['def'] if x['def'] != 0 else 0)
            dataframes[team]['oppda_coef'] = \
                dataframes[team]['ppda_allowed'].apply(lambda x: x['att']/x['def'] if x['def'] != 0 else 0)

        cols_to_sum = ['xG', 'xGA', 'npxG', 'npxGA', 'deep', 'deep_allowed', 'scored', 'missed', 'xpts', 
                       'wins', 'draws', 'loses', 'pts', 'npxGD']
        cols_to_mean = ['ppda_coef', 'oppda_coef']

        frames = []
        for team, df in dataframes.items():
            sum_data = pd.DataFrame(df[cols_to_sum].sum()).transpose()
            mean_data = pd.DataFrame(df[cols_to_mean].mean()).transpose()
            final_df = sum_data.join(mean_data)
            final_df['team'] = team
            final_df['matches'] = len(df)
            frames.append(final_df)

        full_stat = pd.concat(frames)

        team_attrs = ['team', 'matches', 'wins', 'draws', 'loses', 'scored', 'missed', 'pts', 'xG',
                      'npxG', 'xGA', 'npxGA', 'npxGD', 'ppda_coef', 'oppda_coef', 'deep', 'deep_allowed', 'xpts']
        full_stat = full_stat[team_attrs]
        full_stat.sort_values('pts', ascending=False, inplace=True)
        full_stat.reset_index(inplace=True, drop=True)
        full_stat['position'] = range(1,len(full_stat)+1)

        full_stat['xG_diff'] = full_stat['xG'] - full_stat['scored']
        full_stat['xGA_diff'] = full_stat['xGA'] - full_stat['missed']
        full_stat['xpts_diff'] = full_stat['xpts'] - full_stat['pts']

        cols_to_int = ['wins', 'draws', 'loses', 'scored', 'missed', 'pts', 'deep', 'deep_allowed']
        full_stat[cols_to_int] = full_stat[cols_to_int].astype(int)

        col_order = ['position', 'team', 'matches', 'wins', 'draws', 'loses', 'scored', 'missed', 
                     'pts', 'xG', 'xG_diff', 'npxG', 'xGA', 'xGA_diff', 'npxGA', 'npxGD', 'ppda_coef', 
                     'oppda_coef', 'deep', 'deep_allowed', 'xpts', 'xpts_diff']
        full_stat = full_stat[col_order]
        full_stat = full_stat.set_index('position')
        # print(full_stat.head(20))

        season_data[season] = full_stat

    df_season = pd.concat(season_data)
    full_data[league] = df_season

data = pd.concat(full_data)
data.head()

team  matches  wins  draws  loses  scored  \
             position                                                         
La_liga 2014 1               Barcelona       38    30      4      4     110   
             2             Real Madrid       38    30      2      6     118   
             3         Atletico Madrid       38    23      9      6      67   
             4                Valencia       38    22     11      5      70   
             5                 Sevilla       38    23      7      8      71   

                       missed  pts          xG    xG_diff  ...        xGA  \
             position                                      ...              
La_liga 2014 1             21   94  102.980152  -7.019848  ...  28.444293   
             2             38   92   95.766243 -22.233757  ...  42.607198   
             3             29   78   57.047670  -9.952330  ...  29.069107   
             4             32   77   55.062500 -14.937500  ...  39.392572   
             5             45   76   69.526624  -1.473376  ...  47.862742   

                       xGA_diff      npxGA      npxGD  ppda_coef  oppda_coef  \
             position                                                          
La_liga 2014 1         7.444293  24.727907  73.049305   5.683535   16.367593   
             2         4.607198  38.890805  47.213090  10.209085   12.929510   
             3         0.069107  26.839271  25.748737   8.982028    9.237091   
             4         7.392572  33.446477  16.257501   8.709827    7.870225   
             5         2.862742  41.916529  20.178070   8.276148    9.477805   

                       deep  deep_allowed     xpts  xpts_diff  
             position                                          
La_liga 2014 1          489           114  94.0813     0.0813  
             2          351           153  81.7489   -10.2511  
             3          197           123  73.1353    -4.8647  
             4          203           172  63.7068   -13.2932  
             5          305           168  67.3867    -8.6133  

[5 rows x 21 columns]

In [6]:
data.to_csv(data_path + 'understat.csv', index=False)

# Fbref.com

Inspired from this [Github](https://github.com/parth1902/Scrape-FBref-data/).

In [7]:
data_path = 'data/fbref/'

if not os.path.isdir(data_path):
    os.mkdir(data_path)

In [8]:
with open('attributes.json', 'r', encoding='utf-8') as f:
    attributes = json.load(f)
    
# standard stats)
stats = attributes['stats']
stats2 = attributes['stats2']

# goalkeeping
keepers = attributes['keepers']
keepers2 = attributes['keepers2']

# advanced goalkeeping 
keepersadv = attributes['keepersadv']
keepersadv2 = attributes['keepersadv2']

# shooting
shooting = attributes['shooting']
shooting2 = attributes['shooting2']
shooting3 = attributes['shooting3']

# passing
passing = attributes['passing']
passing2 = attributes['passing2']

# passtypes
passing_types = attributes['passing_types']
passing_types2 = attributes['passing_types2']

# goal and shot creation
gca = attributes['gca']
gca2 = attributes['gca2']

# defensive actions
defense = attributes['defense']
defense2 = attributes['defense2']

# possession
possession = attributes['possession']
possession2 = attributes['possession2']

# playingtime
playingtime = attributes['playingtime']
playingtime2 = attributes['playingtime2']

# miscallaneous
misc = attributes['misc']
misc2 = attributes['misc2']

In [9]:
# Function to get the tables in a page

def get_tables(url):
    res = requests.get(url)
    ## The next two lines get around the issue with comments breaking the parsing.
    comm = re.compile("<!--|-->")
    soup = BeautifulSoup(comm.sub("", res.text), 'lxml')
    all_tables = soup.findAll("tbody")
    
    team_table = all_tables[0]
    player_table = all_tables[1]
    
    return player_table, team_table

In [10]:
# Functions to convert the tables' content into a pandas Dataframe

def get_frame(features, player_table, big5):
    pre_df_player = dict()
    features_wanted_player = features
    rows_player = player_table.find_all('tr')
    
    for row in rows_player:
        if(row.find('th', {"scope":"row"}) != None):
            if big5:
                league = row.find('td', {"data-stat":"comp_level"}).text.strip().encode().decode("utf-8")
                league = ' '.join(league.split()[1:])
                if 'league' in pre_df_player:
                    pre_df_player['league'].append(league)
                else:
                    pre_df_player['league'] = [league]
                
    
            for f in features_wanted_player:
                cell = row.find("td", {"data-stat": f})
                if cell is None:
                    text = None
                else:
                    a = cell.text.strip().encode()
                    text = a.decode("utf-8")
                    if(text == ''):
                        text = '0'

                    if((f != 'player') & (f != 'nationality') & (f != 'position') & (f != 'squad') & (f != 'age') \
                       &(f != 'birth_year')):
                        text = float(text.replace(',',''))
                
                if f in pre_df_player:
                    pre_df_player[f].append(text)
                else:
                    pre_df_player[f] = [text]
                    
    df_player = pd.DataFrame.from_dict(pre_df_player)
    
    return df_player


def get_frame_team(features, team_table, big5):
    pre_df_squad = dict()
    #Note: features does not contain squad name, it requires special treatment
    features_wanted_squad = features
    rows_squad = team_table.find_all('tr')
    
    squad_tag = 'td' if big5 else 'th'
    
    for row in rows_squad:
        if(row.find('th', {"scope":"row"}) != None and row.find(squad_tag,{"data-stat":"squad"}) != None):
            name = row.find(squad_tag,{"data-stat":"squad"}).text.strip().encode().decode("utf-8")
            if 'squad' in pre_df_squad:
                pre_df_squad['squad'].append(name)
            else:
                pre_df_squad['squad'] = [name]
            
            if big5:
                league = row.find('td', {"data-stat":"comp"}).text.strip().encode().decode("utf-8")
                league = ' '.join(league.split()[1:])
                if 'league' in pre_df_squad:
                    pre_df_squad['league'].append(league)
                else:
                    pre_df_squad['league'] = [league]
            
            for f in features_wanted_squad:
                cell = row.find("td", {"data-stat": f})
                if cell is None:
                    text = None
                else:
                    a = cell.text.strip().encode()
                    text = a.decode("utf-8")
                    if(text == ''):
                        text = '0'
                    if((f!='player') & (f!='nationality') & (f!='position') & (f!='squad') & (f!='age') \
                       & (f!='birth_year')):
                        text = float(text.replace(',',''))
                if f in pre_df_squad:
                    pre_df_squad[f].append(text)
                else:
                    pre_df_squad[f] = [text]
    
    df_squad = pd.DataFrame.from_dict(pre_df_squad)
    
    return df_squad


def frame_for_category(category, top, end, features, big5):
    url = (top + category + end)
    player_table, team_table = get_tables(url)
    df_player = get_frame(features, player_table, big5)
    
    return df_player


def frame_for_category_team(category, top, end, features, big5):
    url = (top + category + end)
    player_table, team_table = get_tables(url)
    if big5:
        df_team = get_frame_team(features, player_table, big5)
    else:
        df_team = get_frame_team(features, team_table, big5)
    
    return df_team

In [11]:
# Function to get the player data for outfield player, includes all categories - standard stats, shooting
# passing, passing types, goal and shot creation, defensive actions, possession, and miscallaneous
def get_outfield_data(top, end, big5):
    attrs = [stats, shooting2, passing2, passing_types2, gca2, defense2, possession2, misc2]
    attrs_names = ['stats', 'shooting', 'passing', 'passing_types', 'gca', 'defense', 'possession', 'misc']
    
    dfs = []
    for i, attr in enumerate(attrs_names):
        dfs.append(frame_for_category(attr, top, end, attrs[i], big5))
    
    df = pd.concat(dfs, axis=1)
    df = df.loc[:,~df.columns.duplicated()]
    
    return df


# Function to get keeping and advance goalkeeping data
def get_keeper_data(top, end, big5):
    df1 = frame_for_category('keepers', top, end, keepers, big5)
    df2 = frame_for_category('keepersadv', top, end, keepersadv2, big5)
    df = pd.concat([df1, df2], axis=1)
    df = df.loc[:, ~df.columns.duplicated()]
    
    return df


# Function to get team-wise data accross all categories as mentioned above
def get_team_data(top, end, big5=False):
    attrs = [stats2, keepers2, keepersadv2, shooting3, passing2, passing_types2, gca2, defense2, possession2, misc2]
    attrs_names = ['stats', 'keepers', 'keepersadv', 'shooting', 'passing', 'passing_types', 'gca', 'defense', 
                  'possession', 'misc']
    
    dfs = []
    for i, attr in enumerate(attrs_names):
        dfs.append(frame_for_category_team(attr, top, end, attrs[i], big5))
    
    df = pd.concat(dfs, axis=1)
    df = df.loc[:,~df.columns.duplicated()]
    
    return df

## Examples

In [12]:
df_outfield = get_outfield_data('https://fbref.com/en/comps/9/','/Premier-League-Stats', big5=False)
df_outfield

,player,nationality,position,squad,age,birth_year,games,games_starts,minutes,goals,...,fouls,fouled,offsides,pens_won,pens_conceded,own_goals,ball_recoveries,aerials_won,aerials_lost,aerials_won_pct
0,Patrick van Aanholt,nl NED,DF,Crystal Palace,30-123,1990,10.0,9.0,829.0,0.0,...,6.0,3.0,3.0,0.0,0.0,0.0,96.0,6.0,7.0,46.2
1,Tammy Abraham,eng ENG,FW,Chelsea,23-089,1997,14.0,8.0,782.0,6.0,...,14.0,10.0,3.0,1.0,0.0,0.0,39.0,32.0,22.0,59.3
2,Che Adams,eng ENG,FW,Southampton,24-170,1996,16.0,16.0,1346.0,4.0,...,15.0,25.0,11.0,0.0,0.0,0.0,58.0,20.0,75.0,21.1
3,Tosin Adarabioyo,eng ENG,DF,Fulham,23-097,1997,11.0,11.0,990.0,0.0,...,6.0,4.0,1.0,0.0,0.0,0.0,100.0,25.0,12.0,67.6
4,Adrián,es ESP,GK,Liverpool,33-362,1987,2.0,2.0,180.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,14.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
453,Andre-Frank Zambo Anguissa,cm CMR,MF,Fulham,25-044,1995,15.0,13.0,1194.0,0.0,...,22.0,12.0,0.0,0.0,0.0,0.0,149.0,18.0,14.0,56.3
454,Andi Zeqiri,ch SUI,DF,Brighton,21-191,1999,1.0,0.0,19.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,50.0
455,Oleksandr Zinchenko,ua UKR,DF,Manchester City,24-015,1996,1.0,0.0,13.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
456,Hakim Ziyech,ma MAR,FW,Chelsea,27-286,1993,7.0,5.0,388.0,1.0,...,6.0,8.0,0.0,0.0,0.0,0.0,40.0,0.0,6.0,0.0


In [13]:
df_keeper = get_keeper_data('https://fbref.com/en/comps/Big5/','/players/Big-5-European-Leagues-Stats', big5=True)
df_keeper

,league,player,nationality,position,squad,age,birth_year,games_gk,games_starts_gk,minutes_gk,...,passes_length_avg_gk,goal_kicks,pct_goal_kicks_launched,goal_kick_length_avg,crosses_gk,crosses_stopped_gk,crosses_stopped_pct_gk,def_actions_outside_pen_area_gk,def_actions_outside_pen_area_per90_gk,avg_distance_def_actions_gk
0,Premier League,Adrián,es ESP,GK,Liverpool,33-362,1987,2.0,2.0,180.0,...,36.0,14.0,50.0,38.7,18.0,2.0,11.1,0.0,0.00,12.3
1,Premier League,Alisson,br BRA,GK,Liverpool,28-089,1992,12.0,12.0,1080.0,...,32.2,66.0,36.4,35.7,72.0,2.0,2.8,13.0,1.08,17.2
2,Ligue 1,Saturnin Allagbé,bj BEN,GK,Dijon,27-038,1993,4.0,4.0,360.0,...,35.1,27.0,40.7,34.9,35.0,0.0,0.0,5.0,1.25,16.4
3,Serie A,Nícolas Andrade,br BRA,GK,Udinese,32-262,1988,2.0,2.0,180.0,...,32.1,9.0,55.6,47.4,16.0,0.0,0.0,0.0,0.00,15.2
4,Premier League,Alphonse Areola,fr FRA,GK,Fulham,27-307,1993,14.0,14.0,1260.0,...,36.7,95.0,66.3,47.9,130.0,19.0,14.6,8.0,0.57,13.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150,La Liga,Iván Villar,es ESP,GK,Celta Vigo,23-174,1997,7.0,7.0,630.0,...,39.2,56.0,46.4,40.8,67.0,5.0,7.5,8.0,1.14,17.2
151,La Liga,Rubén Yáñez,es ESP,GK,Getafe,27-079,1993,2.0,2.0,180.0,...,55.7,12.0,83.3,61.2,18.0,2.0,11.1,1.0,0.50,13.5
152,Bundesliga,Robin Zentner,de GER,GK,Mainz 05,26-063,1994,13.0,13.0,1170.0,...,34.8,106.0,55.7,47.3,130.0,10.0,7.7,12.0,0.92,14.0
153,Bundesliga,Ron-Robert Zieler,de GER,GK,Köln,31-322,1989,1.0,0.0,51.0,...,35.0,6.0,83.3,49.7,10.0,0.0,0.0,0.0,0.00,11.8


In [14]:
df_team_big5 = get_team_data('https://fbref.com/en/comps/Big5/','/squads/Big-5-European-Leagues-Stats', big5=True)
df_team_big5

,squad,league,players_used,possession,games,games_starts,minutes,goals,assists,pens_made,...,fouls,fouled,offsides,pens_won,pens_conceded,own_goals,ball_recoveries,aerials_won,aerials_lost,aerials_won_pct
0,vs Alavés,La Liga,25.0,61.0,15.0,165.0,1350.0,17.0,10.0,1.0,...,187.0,193.0,54.0,1.0,2.0,0.0,1483.0,231.0,367.0,38.6
1,vs Angers,Ligue 1,27.0,54.3,17.0,187.0,1530.0,26.0,20.0,2.0,...,241.0,196.0,18.0,2.0,2.0,0.0,1526.0,263.0,222.0,54.2
2,vs Arminia,Bundesliga,24.0,55.2,13.0,143.0,1170.0,23.0,18.0,2.0,...,184.0,158.0,31.0,3.0,0.0,1.0,1327.0,299.0,297.0,50.2
3,vs Arsenal,Premier League,25.0,48.3,16.0,176.0,1440.0,16.0,11.0,0.0,...,187.0,152.0,18.0,1.0,3.0,0.0,1340.0,255.0,202.0,55.8
4,vs Aston Villa,Premier League,19.0,50.3,14.0,154.0,1260.0,14.0,10.0,0.0,...,230.0,159.0,14.0,1.0,4.0,1.0,1250.0,270.0,330.0,45.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
191,Werder Bremen,Bundesliga,25.0,43.8,13.0,143.0,1170.0,15.0,12.0,3.0,...,162.0,187.0,26.0,2.0,3.0,1.0,1183.0,221.0,266.0,45.4
192,West Brom,Premier League,25.0,38.9,16.0,176.0,1440.0,10.0,6.0,0.0,...,175.0,178.0,33.0,0.0,4.0,3.0,1454.0,283.0,315.0,47.3
193,West Ham,Premier League,22.0,41.3,16.0,176.0,1440.0,21.0,16.0,0.0,...,175.0,162.0,31.0,0.0,4.0,0.0,1439.0,325.0,295.0,52.4
194,Wolfsburg,Bundesliga,25.0,51.7,13.0,143.0,1170.0,20.0,16.0,1.0,...,167.0,143.0,30.0,1.0,3.0,1.0,1383.0,232.0,231.0,50.1


## Squad Stats (2000-2021)

In [15]:
years = list(range(2000, 2022))
dfs = []
seasons = []

for i in range(len(years) - 1):
    season = f'{years[i]}-{years[i+1]}'
    
    print(season)
    
    seasons.append(season)
    current_season = season == '2020-2021'
    
    if current_season:
        season = ''
        
    end_prefix = f'/squads/{season}-' if not current_season else '/squads/'
    top_suffix = season if current_season else f'{season}/'
    
    df_team_season = get_team_data(f'https://fbref.com/en/comps/Big5/{top_suffix}',
                                   f'{end_prefix}Big-5-European-Leagues-Stats', 
                                   big5=True)
    
    df_team_season['season'] = seasons[-1]
    
    dfs.append(df_team_season)
    
df_team = pd.concat(dfs, axis=0).set_index(['season', 'league'])
df_team

2000-2001
2001-2002
2002-2003
2003-2004
2004-2005
2005-2006
2006-2007
2007-2008
2008-2009
2009-2010
2010-2011
2011-2012
2012-2013
2013-2014
2014-2015
2015-2016
2016-2017
2017-2018
2018-2019
2019-2020
2020-2021


squad  players_used  possession  games  \
season    league                                                            
2000-2001 Bundesliga      vs 1860 Munich          26.0         0.0   34.0   
          La Liga              vs Alavés          22.0         0.0   38.0   
          Premiership         vs Arsenal          26.0         0.0   38.0   
          Premiership     vs Aston Villa          22.0         0.0   38.0   
          Serie A            vs Atalanta          24.0         0.0   34.0   
...                                  ...           ...         ...    ...   
2020-2021 Bundesliga       Werder Bremen          25.0        43.8   13.0   
          Premier League       West Brom          25.0        38.9   16.0   
          Premier League        West Ham          22.0        41.3   16.0   
          Bundesliga           Wolfsburg          25.0        51.7   13.0   
          Premier League          Wolves          21.0        47.8   16.0   

                          games_starts  minutes  goals  assists  pens_made  \
season    league                                                             
2000-2001 Bundesliga               0.0      0.0    0.0      0.0        0.0   
          La Liga                  0.0      0.0    0.0      0.0        0.0   
          Premiership              0.0      0.0    0.0      0.0        0.0   
          Premiership              0.0      0.0    0.0      0.0        0.0   
          Serie A                  0.0      0.0    0.0      0.0        0.0   
...                                ...      ...    ...      ...        ...   
2020-2021 Bundesliga             143.0   1170.0   15.0     12.0        3.0   
          Premier League         176.0   1440.0   10.0      6.0        0.0   
          Premier League         176.0   1440.0   21.0     16.0        0.0   
          Bundesliga             143.0   1170.0   20.0     16.0        1.0   
          Premier League         176.0   1440.0   15.0      8.0        1.0   

                          pens_att  ...  fouls  fouled  offsides  pens_won  \
season    league                    ...                                      
2000-2001 Bundesliga           0.0  ...    0.0     0.0       0.0       0.0   
          La Liga              0.0  ...    0.0     0.0       0.0       0.0   
          Premiership          0.0  ...    0.0     0.0       0.0       0.0   
          Premiership          0.0  ...    0.0     0.0       0.0       0.0   
          Serie A              0.0  ...    0.0     0.0       0.0       0.0   
...                            ...  ...    ...     ...       ...       ...   
2020-2021 Bundesliga           3.0  ...  162.0   187.0      26.0       2.0   
          Premier League       0.0  ...  175.0   178.0      33.0       0.0   
          Premier League       0.0  ...  175.0   162.0      31.0       0.0   
          Bundesliga           2.0  ...  167.0   143.0      30.0       1.0   
          Premier League       1.0  ...  202.0   165.0      20.0       1.0   

                          pens_conceded  own_goals  ball_recoveries  \
season    league                                                      
2000-2001 Bundesliga                0.0        0.0             None   
          La Liga                   0.0        0.0             None   
          Premiership               0.0        0.0             None   
          Premiership               0.0        0.0             None   
          Serie A                   0.0        0.0             None   
...                                 ...        ...              ...   
2020-2021 Bundesliga                3.0        1.0           1183.0   
          Premier League            4.0        3.0           1454.0   
          Premier League            4.0        0.0           1439.0   
          Bundesliga                3.0        1.0           1383.0   
          Premier League            4.0        2.0           1301.0   

                         aerials_won aerials_lost aerials_won_pct  
season    l

In [19]:
df_team = df_team.sort_index()
df_team.to_csv(data_path + 'squad_stats.csv', index=False)

## Outfield Player stats (2000-2021)

In [20]:
years = list(range(2000, 2022))
dfs_outfield = []
seasons = []

for i in range(len(years) - 1):
    season = f'{years[i]}-{years[i+1]}'
    
    print(season)
    
    seasons.append(season)
    current_season = season == '2020-2021'
    
    if current_season:
        season = ''
        
    end_prefix = f'/players/{season}-' if not current_season else '/players/'
    top_suffix = season if current_season else f'{season}/'
    
    df_outfield_season = get_outfield_data(f'https://fbref.com/en/comps/Big5/{top_suffix}',
                                       f'{end_prefix}Big-5-European-Leagues-Stats', 
                                       big5=True)
    
    df_outfield_season['season'] = seasons[-1]
    
    dfs_outfield.append(df_outfield_season)
    
df_outfield = pd.concat(dfs_outfield, axis=0).set_index(['season', 'league']).sort_index()
df_outfield

2000-2001
2001-2002
2002-2003
2003-2004
2004-2005
2005-2006
2006-2007
2007-2008
2008-2009
2009-2010
2010-2011
2011-2012
2012-2013
2013-2014
2014-2015
2015-2016
2016-2017
2017-2018
2018-2019
2019-2020
2020-2021


player nationality position           squad  \
season    league                                                              
2000-2001 Bundesliga         Otto Addo      gh GHA       MF        Dortmund   
          Bundesliga           Adhemar      br BRA       FW       Stuttgart   
          Bundesliga      Silvio Adzic      de GER    FW,MF  Kaiserslautern   
          Bundesliga      Victor Agali      ng NGA       FW   Hansa Rostock   
          Bundesliga  Jan Åge Fjørtoft      no NOR       FW  Eint Frankfurt   
...                                ...         ...      ...             ...   
2020-2021 Serie A          Simone Zaza      it ITA       FW          Torino   
          Serie A     Marvin Zeegelaar      nl NED       DF         Udinese   
          Serie A      Piotr Zieliński      pl POL       MF          Napoli   
          Serie A          Jeroen Zoet      nl NED       GK          Spezia   
          Serie A        Filip Đuričić      rs SRB    FW,MF        Sassuolo   

                         age birth_year  games  games_starts  minutes  goals  \
season    league                                                               
2000-2001 Bundesliga      25       1975   32.0          24.0   2268.0    9.0   
          Bundesliga      28       1972   11.0           8.0    660.0    7.0   
          Bundesliga      19       1980    6.0           1.0    162.0    1.0   
          Bundesliga      21       1978   22.0          21.0   1850.0    5.0   
          Bundesliga      33       1967   14.0           7.0    734.0    3.0   
...                      ...        ...    ...           ...      ...    ...   
2020-2021 Serie A     29-188       1991    7.0           6.0    464.0    1.0   
          Serie A     30-140       1990    9.0           9.0    702.0    0.0   
          Serie A     26-224       1994   11.0           8.0    696.0    1.0   
          Serie A     29-359       1991    2.0           2.0    153.0    0.0   
          Serie A     28-335       1992   11.0          10.0    763.0    3.0   

                      ...  fouls  fouled  offsides  pens_won  pens_conceded  \
season    league      ...                                                     
2000-2001 Bundesliga  ...    0.0     0.0       0.0       0.0            0.0   
          Bundesliga  ...    0.0     0.0       0.0       0.0            0.0   
          Bundesliga  ...    0.0     0.0       0.0       0.0            0.0   
          Bundesliga  ...    0.0     0.0       0.0       0.0            0.0   
          Bundesliga  ...    0.0     0.0       0.0       0.0            0.0   
...                   ...    ...     ...       ...       ...            ...   
2020-2021 Serie A     ...   14.0    13.0       1.0       0.0            0.0   
          Serie A     ...   14.0     3.0       1.0       0.0            0.0   
          Serie A     ...    4.0     9.0       0.0       0.0            0.0   
          Serie A     ...    2.0     0.0       0.0       0.0            1.0   
          Serie A     ...   13.0    22.0       3.0       0.0            0.0   

                      own_goals  ball_recoveries  aerials_won  aerials_lost  \
season    league                                                              
2000-2001 Bundesliga        0.0             None         None          None   
          Bundesliga        0.0             None         None          None   
          Bundesliga        0.0             None         None          None   
          Bundesliga        0.0             None         None          None   
          Bundesliga        0.0             None         None          None   
...                         ...              ...          ...           ...   
2020-2021 Serie A           0.0             18.0         12.0          25.0   
          Serie A           0.0             62.0         17.0          11.0   
          Serie A           0.0             48.0          1.0           5.0   
          Serie A           0.0             10.0          0.0     

In [21]:
df_outfield.to_csv(data_path + 'outfield_player_stats.csv', index=False)

## Keepers stats (2000-2021)

In [22]:
years = list(range(2000, 2022))
dfs_keepers = []
seasons = []

for i in range(len(years) - 1):
    season = f'{years[i]}-{years[i+1]}'
    
    print(season)
    
    seasons.append(season)
    current_season = season == '2020-2021'
    
    if current_season:
        season = ''
        
    end_prefix = f'/players/{season}-' if not current_season else '/players/'
    top_suffix = season if current_season else f'{season}/'
    
    df_keepers_season = get_keeper_data(f'https://fbref.com/en/comps/Big5/{top_suffix}',
                                     f'{end_prefix}Big-5-European-Leagues-Stats', 
                                     big5=True)
    
    df_keepers_season['season'] = seasons[-1]
    
    dfs_keepers.append(df_keepers_season)
    
df_keeper = pd.concat(dfs_keepers, axis=0).set_index(['season', 'league']).sort_index()
df_keeper

2000-2001
2001-2002
2002-2003
2003-2004
2004-2005
2005-2006
2006-2007
2007-2008
2008-2009
2009-2010
2010-2011
2011-2012
2012-2013
2013-2014
2014-2015
2015-2016
2016-2017
2017-2018
2018-2019
2019-2020
2020-2021


player nationality position          squad  \
season    league                                                                
2000-2001 Bundesliga       Alexander Bade      de GER       GK           Köln   
          Bundesliga       Hans-Jörg Butt      de GER       GK   Hamburger SV   
          Bundesliga         Bernd Dreher      de GER       GK  Bayern Munich   
          Bundesliga  Rein van Duijnhoven      nl NED       GK         Bochum   
          Bundesliga         Thomas Ernst      de GER       GK      Stuttgart   
...                                   ...         ...      ...            ...   
2020-2021 Serie A        Marco Sportiello      it ITA       GK       Atalanta   
          Serie A        Thomas Strakosha      al ALB       GK          Lazio   
          Serie A       Wojciech Szczęsny      pl POL       GK       Juventus   
          Serie A      Ciprian Tătărușanu      ro ROU       GK          Milan   
          Serie A             Jeroen Zoet      nl NED       GK         Spezia   

                         age birth_year  games_gk  games_starts_gk  \
season    league                                                     
2000-2001 Bundesliga      29       1970       5.0              5.0   
          Bundesliga      26       1974      32.0             32.0   
          Bundesliga      33       1966       1.0              1.0   
          Bundesliga      32       1967      33.0             33.0   
          Bundesliga      32       1967       2.0              2.0   
...                      ...        ...       ...              ...   
2020-2021 Serie A     28-234       1992       8.0              7.0   
          Serie A     25-286       1995       5.0              5.0   
          Serie A     30-256       1990       9.0              9.0   
          Serie A     34-325       1986       1.0              1.0   
          Serie A     29-359       1991       2.0              2.0   

                      minutes_gk  goals_against_gk  ...  passes_length_avg_gk  \
season    league                                    ...                         
2000-2001 Bundesliga       450.0               7.0  ...                   0.0   
          Bundesliga      2803.0              52.0  ...                   0.0   
          Bundesliga        90.0               0.0  ...                   0.0   
          Bundesliga      2918.0              64.0  ...                   0.0   
          Bundesliga       180.0               2.0  ...                   0.0   
...                          ...               ...  ...                   ...   
2020-2021 Serie A          655.0              15.0  ...                  30.7   
          Serie A          450.0              11.0  ...                  31.6   
          Serie A          810.0              11.0  ...                  27.6   
          Serie A           90.0               3.0  ...                  21.8   
          Serie A          153.0               4.0  ...                  33.0   

                      goal_kicks  pct_goal_kicks_launched  \
season    league                                            
2000-2001 Bundesliga         0.0                      0.0   
          Bundesliga         0.0                      0.0   
          Bundesliga         0.0                      0.0   
          Bundesliga         0.0                      0.0   
          Bundesliga         0.0                      0.0   
...                          ...                      ...   
2020-2021 Serie A           37.0                     59.5   
          Serie A           31.0                     16.1   
          Serie A           51.0                     15.7   
          Serie A            4.0                     50.0   
          Serie A            8.0                     25.0   

                      goal_kick_length_avg  crosses_gk  crosses_stopped_gk  \
season    league                                                             
2000-2001 Bundesliga                   0.0         0.0            

In [23]:
df_keeper.to_csv(data_path + 'keepers_stats.csv', index=False)

## Match logs